In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Diretório do script
HERE = sys.path[0]

In [4]:
train_path = os.path.join(HERE, '..', 'cell_dataset', 'train')
test_path = os.path.join(HERE, '..', 'cell_dataset', 'test')

In [19]:
batch_size = 50
color_mode = 'rgb'
image_shape = (100,100, 3)

In [21]:
train_image_gen = image_dataset_from_directory(train_path,
                                                image_size=image_shape[:2],
                                                color_mode=color_mode,
                                                batch_size=batch_size)

Found 6000 files belonging to 6 classes.


In [22]:
test_image_gen = image_dataset_from_directory(test_path,
                                                image_size=image_shape[:2],
                                                color_mode=color_mode,
                                                batch_size=batch_size)

Found 1200 files belonging to 6 classes.


In [23]:
#https://stats.stackexchange.com/questions/148139/rules-for-selecting-convolutional-neural-network-hyperparameters
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary so we use sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
results = model.fit_generator(train_image_gen,epochs=20,
                              validation_data=test_image_gen)

Epoch 1/20
120/120 [==============================] - 47s 380ms/step - loss: -17629181952.0000 - accuracy: 0.1677 - val_loss: -138196353024.0000 - val_accuracy: 0.1667
Epoch 2/20
120/120 [==============================] - 34s 285ms/step - loss: -3120649994240.0000 - accuracy: 0.1667 - val_loss: -11465229598720.0000 - val_accuracy: 0.1667
Epoch 3/20
 21/120 [====>.........................] - ETA: 28s - loss: -15131222212608.0000 - accuracy: 0.1714

KeyboardInterrupt: 